In [6]:
import requests
import time
from bs4 import BeautifulSoup
from datetime import date

# Request newegg data
newegg_url = 'https://www.newegg.com/p/pl?N=100007709%204814&page='
page_size = '&PageSize=96'
filename = './csv/newegg_GPU_' + str(date.today()) + '.csv'
header = 'Date, Product_name, Producer, Price, Shipping, Rating, Number of Ratings\n'

In [8]:
file = open(filename, 'w')
file.write(header)
# loop over many pages, about 10, max is about 27 pages
for j in range(1,28):
    r = requests.get(newegg_url + str(j) + page_size)
    newegg_soup = BeautifulSoup(r.text, 'html.parser')
    # get list of video cards
    vCard_list = newegg_soup.findAll('div', {'class':'item-container'})
    for vCard in vCard_list:
        # get video card name and brand
        vCard_title = vCard.findAll('a', {'class':'item-title'})
        try:
            if len(vCard_title):
                vCard_name = vCard_title[0].text.split(' ', 1)[1]
                vCard_brand = vCard_title[0].text.split(' ')[0]
            else:
                vCard_img = vCard.findAll('a', {'class':'item-brand'})
                if len(vCard_img):
                    vCard_brand = vCard_img[0]['title']
                else:
                    vCard_name = ''
                    vCard_brand = ''
        except:
            vCard_name = ''
            vCard_brand = ''
        
        # get product ratings
        vCard_rating = vCard.findAll('a', {'class':'item-rating'})
        try:
            if len(vCard_rating) and str(vCard_rating[0]).find('title='):
                x = str(vCard_rating[0]).find('title=')
                vCard_rate5 = vCard_rating[0]['title']
                vCard_n_ratings = str(vCard_rating[0].span.text).strip('()')
        except:
            vCard_rate5 = ''
            vCard_n_ratings = ''

        # get product price
        vCard_cost = vCard.findAll('li', {'class':'price-current'})
        vCard_shipping = vCard.findAll('li', {'class':'price-ship'})
        try:
            if len(vCard_cost):
                vCard_price = vCard_cost[0].strong.text
            if len(vCard_shipping):
                vCard_ship_price = vCard_shipping[0].text.strip()
        except:
            vCard_price = ''
            vCard_ship_price = ''

        file.write(str(date.today()) + ',' + vCard_name.replace(',', '') + ',' + vCard_brand + ',' 
                   + vCard_price.replace(',', '') + ',' + vCard_ship_price + ',' + vCard_rate5 + ',' + vCard_n_ratings + '\n')
    time.sleep(10)
file.close()

KeyboardInterrupt: 

In [7]:
header = 'Brand,Series,Model,Interface,Chipset Manufacturer,GPU Series,GPU,Core Clock,Boost Clock,Effective Memory Clock,Memory Size,Memory Interface,Memory Type,DirectX,OpenGL,HDMI,DisplayPort,Max Resolution,SLI Support,Thermal Design Power,Date First Available'
header_list = header.split(',')
file = open('temp.csv', 'w')
file.write(header+'\n')
for j in range(1, 2):
    # get request and make soup
    r1 = requests.get(newegg_url + str(j) + page_size)
    bs = BeautifulSoup(r1.text, 'html.parser')
    print()
    # list all gpus
    gpu_list = bs.findAll('div', {'class':'item-container'})
    for gpu in gpu_list:
        product_url = gpu.findAll('a', {'class':'item-title'})[0]['href']
        
        
        # get request and make soup for product page
        r2 = requests.get(product_url)
        bs2 = BeautifulSoup(r2.text, 'html.parser')
        
        # find specs
        #if()
        bs2_details = bs2.findAll('div', {'id':'Specs', 'class':'plinks'})[0].findAll('fieldset')
        print('working')
        # dictionary of spec data
        dic = {}
        for specs in bs2_details:
            try:
                for dl in specs.findAll('dl'):
                    dic.update({str(dl.dt.text).replace(',', ''):str(dl.dd.text).replace(',', '')})
            except:
                print('nothing')
        record = ''
        for index in header_list:
            if index in dic.keys():
                record = record + dic[index] + ','
            else:
                record = record + ','
        file.write(record[0:len(record)-1] + '\n')
file.close()